<a href="https://colab.research.google.com/github/Vaselefth/Neural_Networks_Project/blob/master/NeuralNetworksProject1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np
import io
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

#Load Dataset (magic04data.csv) on Colab
"""
from google.colab import files

uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
"""
#Transform csv file to Dataframe with Pandas, for manipulation and check for missing values
missing_values = ["n/a", "na", "--","null"]
dataframe = pd.read_csv(io.StringIO(uploaded['magic04data.csv'].decode('utf-8')),header=None, na_values = missing_values)
dataframe.dropna(how='any')
number_of_missing_values = dataframe.isnull().sum().sum()
print("Missing values in dataset are: " + str(number_of_missing_values) + "\n")

#Replace Classification Values at column 10 with numeric values 0 for "g" and 1 for "h"
dataframe[10].replace({"g": 0, "h": 1}, inplace=True)

#Shuffle Dataset because Classes were ordered 
df = dataframe.sample(frac=1).reset_index(drop=True)

#Use Data Standardization for every input column with this method {y = (x - mean) / standard_deviation}
scaler = StandardScaler()
for i in range(10):
  df[i] = scaler.fit_transform(df[i].values.reshape(-1,1))

#Get some Test Data for later
Test_Inputs = []
Test_Class = []
for i in range(10):
  Test_Inputs.append(list(df.iloc[i,:-1]))
  Test_Class.append(df.iloc[i, -1])

#Drop the previous Test Data from Dataset so that cant be memorized and use only for testing
for i in range(10):
  df.drop(i,inplace=True)

#Split Dataset into Input (X) and Output (y) variables
X = df.loc[:, 0:9]
y = df.loc[:, 10]

#Split into train and test datasets with 1/3 test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

print("Final Dataset Used:" + "\n")
print(df)

#Define the Neural Network Model 
model = Sequential()  
model.add(Dense(20, input_dim=10, activation='relu'))
#model.add(Dense(30, activation='relu'))
#model.add(Dense(30, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

#Compile the Model and calculate accuracy metric
opt = SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#Fit the Model and return history object for more details
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32)

#Evaluate the Model
_, train_accuracy = model.evaluate(X_train, y_train, verbose=0)
_, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Train Accuracy: %.2f, Test_Accuracy: %.2f' % (train_accuracy*100, test_accuracy*100))
print("\n")

#Test the Neural Network to new Data
Tests = np.array(Test_Inputs) 
Class = np.array(Test_Class) 
predictions = model.predict_classes(Tests)
print("Test examples:" + "\n")
for i in range(10):
	print('%s => %d (expected %d)' % (Tests[i].tolist(), predictions[i], Class[i]))

#Plot loss during training
plt.subplot(211)
plt.title('Loss')
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Test')
plt.legend()

#Plot accuracy during training
plt.subplot(212)
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Test')
plt.legend()
plt.show()

